In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import TfidfVectorizer
#função para pré processamento de textos

from nltk.corpus import stopwords
import nltk
import tqdm

In [ ]:
!pip3 install -q kaggle

In [ ]:
kjson = pd.read_json('kaggle.json', typ='series')

In [ ]:
os.environ['KAGGLE_USERNAME']= "emillyvictoria"
os.environ['KAGGLE_KEY']= kjson.key

In [ ]:
!kaggle datasets download --force tboyle10/medicaltranscriptions

 21% 1.00M/4.85M [00:00<00:00, 6.92MB/s]
100% 4.85M/4.85M [00:00<00:00, 23.8MB/s]


In [ ]:
!unzip medicaltranscriptions.zip

Archive:  medicaltranscriptions.zip
  inflating: mtsamples.csv           


In [ ]:
df = pd.read_csv('/content/mtsamples.csv')
df.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [ ]:
df = df.groupby("medical_specialty").filter(lambda x: len(x) > 200)

In [ ]:
df= df[['transcription', 'medical_specialty']]
df = df.drop(df[df['transcription'].isna()].index)

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
#y = pd.get_dummies(df.medical_specialty)

In [ ]:
x = df.transcription

from sklearn.model_selection import train_test_split
Xtrain, xval, ytrain, yval = train_test_split(x,y , test_size = 0.2, random_state=42)

In [ ]:
vec = TfidfVectorizer(min_df=5, lowercase=True, ngram_range=(1,2))

Xtrain = vec.fit_transform(Xtrain)
xval = vec.transform(xval)

#BERT

In [ ]:
#faz o pré processamento de uma forma mais moderna
#vamos usá-lo para fazer uma 'tokenização', ou seja, a codificação de textos 

In [ ]:
!pip install transformers -q

In [ ]:
import torch
import transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [ ]:
model_class = transformers.DistilBertModel
tokenizer_class = transformers.DistilBertTokenizer
pretrained_weights = 'distilbert-base-uncased'

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenized = df['transcription'].apply((lambda x: tokenizer.encode(x[:200], add_special_tokens=True)))

In [ ]:
tokenized

0       [101, 1016, 1011, 1040, 1049, 1011, 5549, 1024...
1       [101, 1015, 1012, 1996, 2187, 18834, 7277, 793...
2       [101, 1016, 1011, 1040, 9052, 11522, 3695, 131...
3       [101, 6412, 1024, 1010, 1015, 1012, 3671, 1505...
4       [101, 1016, 1011, 1040, 2817, 1010, 1015, 1012...
                              ...                        
3304    [101, 12407, 1024, 1010, 3108, 3255, 1012, 101...
3305    [101, 2708, 12087, 1024, 1010, 3108, 3255, 101...
3306    [101, 2381, 1997, 2556, 7355, 1024, 1010, 1996...
3307    [101, 2381, 1997, 2556, 7355, 1024, 1010, 2720...
3308    [101, 3114, 2005, 16053, 1024, 1010, 19470, 90...
Name: transcription, Length: 3309, dtype: object

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

#esse código visualiza linha por linha para descobrir qual é a maior linha, ou seja, a maior lista de códigos
#o padded preenche espaços vazios com zeros para todas as linhas terem o mesmo shape 

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape
#visualiza oq é padded e oq não é

(3309, 88)

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    features = model(input_ids, attention_mask=attention_mask)
#cria tensores com o padded e o attention_mask para transformar em features

In [ ]:
features = features[0][:,0,:].numpy()

In [ ]:
y = df['encoded_labels']
X = features

KeyError: ignored

#Logistic Regression

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(Xtrain, ytrain)

In [ ]:
pred = lr_clf.predict(Xval)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(yval, pred))

#Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1)

rf.fit(Xtrain, ytrain)
pred = rf.predict(Xval)

In [ ]:
print(classification_report(yval, pred))

#SVM

In [ ]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(Xtrain, ytrain)

pred = svc.predict(Xval)

In [ ]:
print(classification_report(yval, pred))